# Neural Networks Project - Comparing Classic Recommender Algorithms with Deep Neural Recommenders
Ruprecht-Karls-Universität Heidelberg<br>
Institut für Computerlinguistik<br>
Seminar: Introduction to Neural Networks and Sequence-to-Sequence Learning<br>
Seminarsleitung: Michael Staniek<br>
Sommersemester 2020<br>
Ufkun-Bayram Menderes<br>

## Introduction - Recommender Systems
We've been all been there before: We want to make ourselves a cozy winter evening and have prepared snacks and warm blankets to watch the movie of the evening, but at times, we are just overwhelmed with the mere options which we can choose from nowadays on platforms like Hulu or Netflix. I myself haven't experienced it, but apparently not so long ago people decided that they wanted to watch a certain movie, went to their localc Blockbuster, rented the DVD of the movie, watched it, (hopefully) enjoyed it and then safely returned it to their Blockbuster shop. With said platforms like Netflix at our availibity today, this whole procedure has become obsolete, since we can easily click on the respective movie/TV show we want to watch. However, with the meriad of choices available, one might end up just plainly clueless about which movie to watch today. This is the exact point where **Recommender Systems** come into play. What Recommenders do is the following: They track our ratings or consumption patterns on websites such as Netflix and the recommend us items that we, based on our recorded choices of movies, TV shows, Albums etc., might also want to watch and even enjoy to the same extent as we did with our previous items. Regarding this, we might now realize that we have encountered the results of recommender systems and their algorithms more often than not when we're using these platforms, but also online shopping websites like Amazon who never fail us to suggest more items to purchase.<br>
In this project, I want to take a closer look at how classical Recommender Algorithms and the newer Neural Recommenders (based on Neural Networks) handle this task.

## Classical Recommender Algorithms
Classical Recommender Algorithms work on the premise of two different ways of Recommending: **Item-based Recommendations** and **Collaborative Filtering**. As the names suggest, the former method makes recommendations by comparing items to each other and takes into account how users have rated those items, while the latter forms recommendations by mainly comparing users to each other.

The following bulk of code was mainly taken from the book *The Ancient of the Numerati* by Ron Zacharski and was slightly adapted and changed at appropriate places


In [1]:
class recommender():
    def __init__(self, data, k=1, metric='pearson', n=5):
        """
        Recommender class which takes user input data and implements various
        functions for both collaborative filtering and item-based filtering

        Parameters
        ----------
        data : dict
            is the dictionary in which our data, i.e. the user ratings, are contained
        k : int
            determines k number of neighbors we are considering for knn calculation,
            default is k=1
        metric : function
            determines the user similarity metric which we are using for 
            collaborative filtering
            The default is 'pearson' metric, other option is cos_similarity
        n : int, optional
            number of top ratings, the default is 5.

        Returns
        -------
        None.

        """
        self.k = k
        self.n = n
        self.username2id = {}
        self.userid2name = {}
        self.productid2name = {}
        self.frequencies = {}
        self.deviations = {}
        self.metric = metric
        if self.metric == 'pearson':
            self.fn = self.pearson
        if self.metric == 'cos_similarity':
            self.fn = self.cos_similarity
        if type(data).__name__ == 'dict':
            self.data = data
            
    def convertProductID2name(self, id):
        """
        Function which converts given product id's into names

        Parameters
        ----------
        id : product id, the key in the dictionary
            id of the respective products

        Returns
        -------
        self.productid_to_name[id], id
            if the product is in the dictionary, return its value
        else simply return id

        """
        #if the key id is in the dictionary
        if id in self.productid2name:
            #then return its value, which is the product name
            return self.productid2name[id]
        #else simply return the id
        else:
            return id
    
    def user_ratings(self, id, n):
        """
        Function which returns n-top items rated by a user

        Parameters
        ----------
        id : user id
            id of users for who we will return ratings for
        n : int
            number of top ratings which will be returned 

        Returns
        -------
        n top ratings

        """
        # user id is the key in our dictionary, its values are the ratings of 
        # the respective users
        # the ratings as values are in themselves again a dictionary with products
        # as keys and their ratings as values
        print("Ratings for: " + self.userid2name[id])
        #s ave those ratings for ever user with an id 
        ratings = self.data[id]
        # print the length of the values
        print(len(ratings))
        # make a list out of key-value pairs in ratings as tuples
        ratings = list(ratings.items())
        # call the upper function, pass the keys of the tuples as function 
        # arguments and make a list comprehension for key-value pairs in the 
        # tuples
        ratings = [(self.convertProductID2name(k), v) for (k, v) in 
                   ratings]
        # sort the list according to the first element of each pair, which is 
        # the rating, reverse the list so we have top values at the beginning
        ratings.sort(key = lambda artist_tuple: artist_tuple[1], reverse=True)
        # slice the list up till index n so only n number of elements will be returned
        ratings = ratings[:n]
        for rating in ratings:
            print("%s\t%i" % (rating[0], rating[1]))
            
    def load_book_db(self, path=''):
        """
        Function which specifically loads the BX books database/dataset

        Parameters
        ----------
        path : TYPE, optional
            DESCRIPTION. The default is ''.
            Function that loads our book database
            Path is where BX book dataset is located
        Each for Loop is for one of the csv-files in our BX-Dump file
        1st for-loop: loads book ratings made by users
        2nd for-loop: loads book information
        3rd for-loop: loads user information

        Returns
        -------
        None.

        """
        # initialize data dict
        self.data = {}
        # initialize counter
        i = 0
        # load book ratings into self.data in read mode with utf-8 encoding
        f = codecs.open(path + "BX-Book-Ratings.csv", 'r', 'utf-8')
        # iterate over each line in the file
        # lines are structured as: user-id, isbn, user ratings
        for line in f:
            i += 1 # increment counter
            # separate each line into fields by splitting at semicolon
            fields = line.split(";")
            # extract user-id
            user = fields[0].strip('"')
            # extract isbn
            book = fields[1].strip('"')
            # extract user rating
            rating = int(fields[2].strip().strip('"'))
            
            # if the uer in the dictionary
            if user in self.data:
                # then return his current ratings
                current_ratings = self.data[user]
            # if not
            else:
               # create empty ratings dictionary
               current_ratings = {}
            # inser ratings as values for book keys in ratings dictionary
            current_ratings[book] = rating
            # assign dict current_ratings as value to key user
            self.data[user] = current_ratings
        f.close()
        # Now load books into self.productid2name
        # Books contains isbn, title, and author among other fields
        f = codecs.open(path + 'BX-Books.csv', 'r', 'utf-8')
        for line in f:
            i += 1 #increment counter
            fields = line.split(";")
            isbn = fields[0].strip('"')
            title = fields[1].strip('"')
            author = fields[2].strip('"')
            title = title + ' by ' + author
            # assign title as value to key isbn
            self.productid2name[isbn] = title
        f.close()
        #
        #  Now load user info into both self.userid2name and
        #  self.username2id
        #
        f = codecs.open(path + 'BX-Users.csv', 'r', 'utf-8')
        for line in f:
            i += 1 #increment counter
            fields = line.split(';')
            user_id = fields[0].strip('"')
            location = fields[1].strip('"')
            # if age field is filled with an entry, hence why fields is > 3
            if len(fields) > 3:
                age = fields[2].strip().strip('"')
            # if not, than enter NULL into age field
            else:
                age = 'NULL'
            # if an entry for age field apparent, enter 
            if age != 'NULL':
                # then concatetenate info into one string
                value = location + '  (age: ' + age + ')'
            else:
                value = location
            # assign value as value for key user id in dict
            self.userid2name[user_id] = value
            # assign user_id as value to key location
            self.username2id[location] = user_id
        f.close()
        print(i)
        
    def compute_deviations(self):
        """
        Computes item deviations in a given dataset 
        
        args: self, no other argument necessary
        
        Functions computes deviations for each item according to the given 
        formula in Chapter 3 of "The ancient art of Numerati"
        
        After function is called, self.deviations will be filled with each item 
        as key, and its values are dictionaries in which keys represent other items 
        and values represent the deviation to that item in float type

        Returns
        -------
        None.

        """
        # for each person in the data :
        #get their ratings
        for ratings in self.data.values():
            #for each item and its rating in tthe ratings dict
            for (item, rating) in ratings.items():
                self.frequencies.setdefault(item, {})
                self.deviations.setdefault(item, {})
                # for each item2 & rating2 in that set of ratings: 
                for(item2, rating2) in ratings.items():
                    # add the difference between the ratings to our computation
                    if item != item2:
                        self.frequencies[item].setdefault(item2, 0)
                        self.deviations[item].setdefault(item2, 0.0)
                        self.frequencies[item][item2] += 1
                        self.deviations[item][item2] += rating - rating2
        for (item, ratings) in self.deviations.items():
            for item2 in ratings:
                ratings[item2] /= self.frequencies[item][item2]
    
        
    def slope_one_recommend(self, user):
        """
        Implementation of slope one algorithm for recommendation and 
        prediction of an item for a given user, for item-based recommendation

        Parameters
        ----------
        user : str
            user for which we will recommend items and predict his ratings
            self.compute_deviations function must be called before calling this 
            function
            

        Returns
        -------
        recommendations : list
            A list containing tuples as elements
            tuple[0] = item which user hasn't rated yet, type: str
            tuple[1] = predicted rating for that unrated item, type: float

        """
        self.compute_deviations()
        recommendations = {}
        frequencies = {}
        # for every item and rating in the user's recommendations
        for (user_item, user_rating) in self.data[user].items():
            # for every item in our dataset that the user didn't rate
            # example. dict_items([('Taylor Swift', 5), ('PSY', 2)])
            for (diff_item, diff_ratings) in self.deviations.items():
                if diff_item not in self.data[user] and \
                   user_item in self.deviations[diff_item]:
                # get the deviations, frequencies for the item that wasn't rated
                # but only if a value for deviations exists and diff_item is not 
                # in user_ratings
                    freq = self.frequencies[diff_item][user_item]
                    # get the frequency of diff_item and user_item, 
                    # key in frequencies is the different item, second dict indice 
                    # is the frequency with user_item
                    recommendations.setdefault(diff_item, 0.0)
                    frequencies.setdefault(diff_item, 0)
                    # add to the running sum representing the numerator
                    # of the formula 
                    # diff_item is key, result of formula is value
                    recommendations[diff_item] += (diff_ratings[user_item] +
                                                   user_rating)* freq
                    # keep a running sum of the frequency of diffitem
                    frequencies[diff_item] += freq
        recommendations = [(self.convertProductID2name(k), v / frequencies[k])
                           for (k, v) in recommendations.items()]
        recommendations.sort(key=lambda artist_tuple: artist_tuple[1], reverse = True)
        return recommendations
    
    
    
    def minkowski_distance(self, rating1, rating2, p):
        """
        Implementation of Minkowski distance formula

        Parameters
        ----------
        rating1 : dict
            dictionary in which the ratings of user1 are specified, 
            passing as argument into function via "users['users']"
        rating2 : dict
            dictionary in which ratings of user2 are specified, same 
            argument passing conventions as above
        p : int
            specifies parameter 'p' in minkowski distance formula
            p = 1 --> Manhattan distance
            p = 2 --> Euclidean distance

        Returns
        -------
        float
            returns the minkowski distance measure between rating1 and rating 2
            according to parameter 'p'

        """
        distance = 0
        common_ratings = False
        for key in rating1:
            if key in rating2:
                distance += pow(abs(rating1[key] - rating2[key]), p)
                common_ratings = True
        if common_ratings == True:
            return pow(distance, 1/p)
        else:
            return 0
            
        
        
    def pearson(self, rating1, rating2):
        """
        Implementation of Pearson correlation coefficient formula
        Calculates similarity of users via ratings ina given dataset

        Parameters
        ----------
        rating1 : dict
            ratings of the first user, access via "users[user]"
        rating2 : dict
            ratings of 2nd user, same accessing conventions when passing 
            in the argument
        function implements pearson similarity metric by comparing commonly rated 
        elements of both users

        Returns
        -------
        float
            returns a float which measures the similarity of two user according 
            to their set of commonly given ratings, ranges from -1 to 1

        """
        sum_xy = 0
        sum_x = 0
        sum_y = 0
        sum_x2 = 0
        sum_y2 = 0
        n = 0
        for key in rating1:
            if key in rating2:
                n += 1
                x = rating1[key]
                y = rating2[key]
                sum_xy += x * y
                sum_x += x
                sum_y += y
                sum_x2 += pow(x, 2)
                sum_y2 += pow(y, 2)
        if n == 0:
            return 0
        # compute denominator
        denominator = (sqrt(sum_x2 - pow(sum_x, 2) / n)
                       * sqrt(sum_y2 - pow(sum_y, 2) / n))
        if denominator == 0:
            return 0
        else:
            return (sum_xy - (sum_x * sum_y) / n) / denominator
    
    def dot(self, rating1, rating2):
        """
        Computes the dot product SPECIFICALLY for our application
        at hand and without using numpy data structures

        Parameters
        ----------
        rating1 : dict
            dict containing user ratings for user 1, must be indiced in the form 
            of 'users[user]' as function argument in order to properly access user 
            ratings
        rating2 : dict
            dict containing user ratings for user 2, same as above for indicing

        Returns
        -------
        dot : float
            returns the dot product for 2 users 

        """
        for key in rating1:
            if key in rating2:
                dot = sum(rating1[key]*rating2.get(key, 0) for key in rating1)
        return dot
    
    def cos_similarity(self, rating1, rating2):
        """
        Implementation of cosine similarity w.r.t to given datastructures at hand
        Measures similarity between two users by measuring ratings

        Parameters
        ----------
        rating1 : dict 
            user ratings for user 1, dict must be indiced and indexing must be 
            passed as function argument in form of (users['user'])
        rating2 : dict
            user ratings for user 2, same as above 

        Returns
        -------
        cosine similarity
            alternative measure for capturing the similarity between two users, 
            ranges from 0 to 1

        """
        num = self.dot(rating1, rating2)
        sum_r1 = []
        sum_r2 = []
        for vals in rating1.values():
            sum_r1.append(pow(vals, 2))
        for vals in rating2.values():
            sum_r2.append(pow(vals, 2))
            denom = sqrt(sum(sum_r1)) * sqrt(sum(sum_r2))
        return num / denom
        
        
    def averages(self, users):
        """
        Computes average ratings assigned by users in a given dataset

        Parameters
        ----------
        users : dict
            Dictionary in which users as keys and their respective ratings for 
            the respective items (dict) as values are stored

        Returns
        -------
        results : dict
            users are keys, their average ratings (float) are values

        """
        results = {}
        for (key, ratings) in users.items():
            results[key] = float(sum(ratings.values())) / len(ratings.values())
        return results
    
    
    
    def item_similarity(self, artist1, artist2, user_ratings):
        """
        Computes the similarity between two items in a given dataset

        Parameters
        ----------
        artist1 : str
            artist1 for which we will compute the similarity with artist2, must be 
            in user_ratings dict
        artist2 : str
            artist1 for which we will compute the similarity with artist2, must be 
            in user_ratings dict    
        user_ratings : dict
            dictionary containing users as keys, their ratings for items 
            as internal dict as values

        Returns
        -------
        float
            similarity between two items

        """
        
        averages = {}
        for (key, ratings) in user_ratings.items():
            #take the values of the internal dict, sum over each value and divide by length of values
            averages[key] = (float(sum(ratings.values()))
                      / len(ratings.values()))
        numerator = 0
        denom1 = 0
        denom2 = 0
        for (user, ratings) in user_ratings.items():
            #if the users rated both artists
            if artist1 in ratings and artist2 in ratings:
                #compute the averages of each user
                avg = averages[user]
                # in ratings dict, artists are the keys and their respective values, 
                # which we access via indexing, are their ratings
                # those ratings for the artists are subtracted by the average of each user
                # and then multiplied
                numerator += (ratings[artist1] - avg) * (ratings[artist2] - avg)
                denom1 += (ratings[artist1]-avg)**2 
                denom2 += (ratings[artist2]-avg)**2
        return numerator / (sqrt(denom1) * sqrt(denom2))
    
    def normalized_rating(self, user:str, item:str):
        """
        Normalizes the rating of an item for a user in a given dataset according 
        to minimum and maximum rating of that dataset

        Parameters
        ----------
        user : str
            a user who has rated the item function argument
        item : str
            item that a user has rated, is within the internal dictionary as key
        

        Returns
        -------
        float
            normalized rating for a user given said item

        """
        global min_rat
        global max_rat
        min_ratings = []
        max_ratings = []
        for (key, rating) in self.data.items():
            key_max = max(rating.keys(), key=(lambda k: rating[k]))
            key_min = min(rating.keys(), key=(lambda k: rating[k]))
            min_ratings.append(rating[key_min])
            max_ratings.append(rating[key_max])
        min_rat = min(min_ratings)
        max_rat = max(max_ratings)
        if item in self.data[user]:
            return (2*((self.data[user][item]-min_rat)) - (max_rat-min_rat)) / (max_rat-min_rat)
            
    
    def denormalized_rating(self, user:str, item:str):
        """
        Returns a normalized rating back into a scaled rating, 
        given respective rating scale
        
        BEWARE!!! This function takes an user and an item and then gives the denormalized
        rating for it, the denormalizer only takes a float value

        Parameters
        ----------
        user : str
            DESCRIPTION.
        norm_rating : float
            DESCRIPTION.

        Returns
        -------
        None.

        """
        return (0.5*((self.normalized_rating(user, item)+1) * (max_rat-min_rat))) + min_rat
        
    
    def denormalize(self, norm_rating:float):
        """
        

        Parameters
        ----------
        norm_rating : float
            DESCRIPTION.

        Returns
        -------
        None.

        """
        return (0.5*((norm_rating+1) * (max_rat-min_rat))) + min_rat

        
    def predict_rating(self, user:str, item:str):
        """
        Predicts the rating of an item from a user from a given dataset

        Parameters
        ----------
        user : str
            user in a given dataset
        item : str
            an item which the user HAS NOT rated yet

        Returns
        -------
        tuple
            tuple[0] = user 
            tuple[1] = item recommended for that user
            tuple[2] = predicted rating for that item

        """
        #initiate numerator and denumerator of formula
        denom = 0
        numerator = 0
        for rated_item in self.data[user]:
            if rated_item in self.data[user]:
                denom += (self.item_similarity(rated_item, item, self.data)
                          *self.normalized_rating(user, rated_item))
                numerator += abs(self.item_similarity(item, rated_item, self.data))
        return (user, item, self.denormalize(denom/numerator))
   
        
        
            
    def nearest_neighbor(self, username):
        """
        Determines nearest neighbors, i.e. users for a user in a given dataset
        for collaborative filtering.
        Default metric is Pearson correlation coefficient, can be changed to other
        metrics when instatiating an object of Recommender class 

        Parameters
        ----------
        username : str
            username of a user in the set of our users
        
        since self.fn == 'pearson', the nearest neighbors will be calculated 
        using pearson coefficient, can be changed if desired

        Returns
        -------
        distances : list
            a list containing all the distances from the user to the other users
            in the total set of users
            Elements are tuples with users (str) as first element and the distances
            to them as floats as second element

        """
        distances = []
        for instance in self.data:
            # measure distance only if users are distinct 
            if instance != username:
                # Pearson metric is applied
                distance = self.fn(self.data[username], self.data[instance])
                distances.append((instance, distance))
        distances.sort(key=lambda artist_tuple: artist_tuple[1], reverse=True)
        return distances
    
    def recommend(self, user:str):
        """
        Recommends a 

        Parameters
        ----------
        user : str
            user in the set of users

        Returns
        -------
        type = list
            returns a list of top n recommendations for the user 

        """
        # create empty dictionary for recommendations
        recommendations = {}
        # compute the nearest neighbors of a specific user
        nearest = self.nearest_neighbor(user)
        # extract ratings for this specific user from self.data dict
        user_ratings = self.data[user]
        # initialize total distance counter
        total_distance = 0.0
        # iterate over k elements, here k=1
        for i in range(self.k):
            total_distance += nearest[i][1]
        # add up the total distance by summing over the ratings of k nearest neighbors
        # compute the contribution of each k neighbor
        for i in range(self.k):
            weight = nearest[i][1] / total_distance
            # extract name of each of the k neighbors
            name = nearest[i][0]
            # extract ratings of each person out of the k neighbors
            neighbor_ratings = self.data[name]
        # for every artist in neighbor ratings
        for artist in neighbor_ratings:
            # if the artist is not in the user_ratings
            if not artist in user_ratings:
                # and if the artist is not already in recommendations dict 
                # i.e. artists that the neigbor rated but the user didn't
                # for key artists, assign the value of its neighbor rating times the weight 
                # it contributes
                if artist not in recommendations:
                    # for key artists, assign the value of its neighbor rating times the weight 
                    # it contributes
                    recommendations[artist] = (neighbor_ratings[artist]*weight)
                else:
                    # if artist already existing, assign neighbor rating + recommendations rating
                    # and multiply it with weight
                    recommendations[artist] = (recommendations[artist]+neighbor_ratings[artist] * weight)
        # make a list out of the key, value pairs of artist and recommendations tuples
        recommendations = list(recommendations.items())
        # convert the product id to names via list comprehension
        recommendations = [(self.convertProductID2name(k), v) for (k, v) in recommendations]
        # sort the list by rating, reverse in order for top ratings to be at the beginning
        recommendations.sort(key=lambda artist_tuple: artist_tuple[1], reverse=True)
        # return n elements
        return recommendations[:self.n]
    
    def nearest_items(self, user:str):
        """
        Recommends an item to a user by recommending the most similar item 
        to highest rated items by a user

        Parameters
        ----------
        user : str
            user for which we will the items most similar to his top rated item
            

        Returns
        -------
        tuple
        tuple[1] = user, str
        tuple[2] = recommendations, list       

        """
        # compute deviations since self.deviations will be useful for 
        # future calculatios
        self.compute_deviations()
        # sort ratings of a user
        top_rated = {item: rating for item, rating in 
                     sorted(self.data[user].items(), key=lambda value: value[1],
                            reverse=True)}
        # discard all elements which are not top rated
        for key, rating in top_rated.items():
            # determine highest rated item(s) and their rating(s)
            max_user_rating = max(top_rated.keys(), key=(lambda k: top_rated[k]))
            max_user_rating = top_rated.get(max_user_rating)
            #finding elements with multiple values
            mult_top_rated = [k for k, v in top_rated.items() if v == max_user_rating]
        # now we find the most similar item for each top rated item of a user
        similarities = []
        for item in mult_top_rated:
            for unrated_item in self.deviations:
                if unrated_item not in top_rated:
                    item_sim = self.item_similarity(item, unrated_item, self.data)
                    similarities.append((item_sim, item, unrated_item))
        item_near = {}
        # we create a dictionary in which we compute the similarities of every
        # top rated item to other top rated items, where top rated items are keys
        # and list of similarity to an item and item itself as tuples are elements
        # of that list
        for sim, item, near_item in similarities:
            if item in item_near:
                item_near[item].append((sim, near_item))
            else:
                item_near[item] = [(sim, near_item)]
        # sort the values according to their similarity, highest similarity 
        # at top
        for key, rating in item_near.items():
            rating.sort(key=lambda sim: sim[0], reverse=True)
        recommendations = [item_near[item][0] for item in item_near]
        # remove duplicate items, remove the duplicate with lower item similarity
        for sim, item in recommendations:
            for sim2, item2 in recommendations:
                if item == item2 and sim < sim2:
                    recommendations.remove((sim, item))
                elif item == item2 and sim > sim2:
                    recommendations.remove((sim2, item2))
        final_recoms = [elems[1] for elems in recommendations]
        return user, (final_recoms)
    
# rec = recommender(users, k=3)
# print(rec.recommend("Veronica"))
#rec.load_book_db('/Users/ubmenderes/Downloads/BX-Dump/')

## Collaborative Filtering in Classic Recommender Systems

In classic recommenders, the task of user-based **Collaborative Filtering** is handled by computations on user data and the associated databases, which are mainly dictionaries/associative arrays. Collaborative Filtering follows the idea that an item may be recommended to a user if one finds out what kind of items other uses liked or also disliked, this is the *collaborative* part of the method. In other words: If I like certain items and another users likes those items the same way I do, I also might like the items that that other user has rated positively but which I have *not rated* yet. One problem that one might already spot is that this requires users to be somewhat honest and serious about their ratings, since dishonesty in the rating behaviour would influence the databases of users and thus would ultimately skew the computations and their results performed on these datasets. Problematic platforms in which this phenomenon could occur are platforms which are subject to heavy user bias such as YouTube, where the like/dislike ratio isn't necessarily a reflection of the content quality but often ends up being a ratio to determine the current likability of a YouTuber. However, on platforms like IMDB, Metacritic, Amazon, Letterboxd etc., collaborative filtering with emphasis on user ratings could work out fairly well since users on these sites tend to be more objective when rating items. Another way of working around this could be alternative implementations which take into account other factors such as genre, production company, production year etc, but I will not focus on these in this notebook and solely work on ratings.<br>

### Similarity measures - Minkowsi Distance, Manhattan Distance and Euclidean Distance
In the class above, we handle this by introducing and applying **similarity measures** which, in collaborative filtering, measure the similarity of one user to other users of a certain platform/dataset. One such measure is the **Minkowski-Distance**, which is based on this formula:
$$d(x,y) = (\sum_{k=1}^n |x_k - y_k|^p)^{\frac{1}{p}}$$
$x$ and $y$, respectively, thus represent two users as variables, and our target is to find the related distance between them.
Below is the specific implementation in the *recommender* class:

In [ ]:
def minkowski_distance(self, rating1, rating2, p):
        """
        Implementation of Minkowski distance formula

        Parameters
        ----------
        rating1 : dict
            dictionary in which the ratings of user1 are specified, 
            passing as argument into function via "users['users']"
        rating2 : dict
            dictionary in which ratings of user2 are specified, same 
            argument passing conventions as above
        p : int
            specifies parameter 'p' in minkowski distance formula
            p = 1 --> Manhattan distance
            p = 2 --> Euclidean distance

        Returns
        -------
        float
            returns the minkowski distance measure between rating1 and rating 2
            according to parameter 'p'

        """
        distance = 0
        common_ratings = False
        for key in rating1:
            if key in rating2:
                distance += pow(abs(rating1[key] - rating2[key]), p)
                common_ratings = True
        if common_ratings == True:
            return pow(distance, 1/p)
        else:
            return 0

The advantage of the Minkowski Distance metric is that two popular distance metrics from linear algebra, namely **Manhattan-Distance** and **Euclidean Distance** are already included in the Minkowski-Distance, since selecting parameter $p=1$ will give us Manhattan Distance, while choosing $p=2$ gives us Euclidean Distance. With this, we can measure the distance between two users, or to be more precise the difference between the ratings they gave to items rated commonly between them.

### Similarity Measures - Cosine Similarity
Another popular metric measuring simialarity between two users is **Cosine Similarity**:
$$cos(x,y) = \frac{x\cdot y}{||x||\cdot ||y||}$$.
Cosine similarity metric maps the similarity value to an interval of $[-1, 1]$, where 1 naturally indicates best similarity (i.e. equality) and -1 indicates complete divergence. Below is the implementation of cosine similarity in the recommender class:

In [ ]:
def cos_similarity(self, rating1, rating2):
        """
        Implementation of cosine similarity w.r.t to given datastructures at hand
        Measures similarity between two users by measuring ratings

        Parameters
        ----------
        rating1 : dict 
            user ratings for user 1, dict must be indiced and indexing must be 
            passed as function argument in form of (users['user'])
        rating2 : dict
            user ratings for user 2, same as above 

        Returns
        -------
        cosine similarity
            alternative measure for capturing the similarity between two users, 
            ranges from -1 to 1

        """
        num = self.dot(rating1, rating2)
        sum_r1 = []
        sum_r2 = []
        for vals in rating1.values():
            sum_r1.append(pow(vals, 2))
        for vals in rating2.values():
            sum_r2.append(pow(vals, 2))
            denom = sqrt(sum(sum_r1)) * sqrt(sum(sum_r2))
        return num / denom

### User similarity measures - Pearson correlation coefficient
The last user similarity measure that I want to introduce here is the **Pearson correlation coefficient**, an apparently well known metric in statistics. Like in the other measures already presented, our users are represented in the form of variables $x$ and $y$, and our task yet again consists in finding out the distance between them, which in this case means measuring the similarity between them by analyzing their ratings. Like cosine similarity, Pearson correlation coefficient maps our similarity value to an intervall of $[-1, 1]$, where 1 means *total positive linear correlation* (i.e. the users and their ratings are practically the same), -1 means *total negative linear correlation* and 0 means no linear correlation whatsoever. The formula for the coefficient, which is also commonly known as *Pearson r* is as follows:
$$r = \frac{\sum^n_{i=1}(x_i - \tilde{x})(y_i-\tilde{y})}{\sqrt{\sum^n_{i=1}}(x_i - \tilde{x})^2 \sqrt{\sum_{i=1}^n}(y_i - \tilde{y})^2}$$
Since this formula is unnecessarily difficult to program, an approximation (which still satisfies the desired objective) is chosen instead.
The implementation:

In [ ]:
def pearson(self, rating1, rating2):
        """
        Implementation of Pearson correlation coefficient formula
        Calculates similarity of users via ratings ina given dataset

        Parameters
        ----------
        rating1 : dict
            ratings of the first user, access via "users[user]"
        rating2 : dict
            ratings of 2nd user, same accessing conventions when passing 
            in the argument
        function implements pearson similarity metric by comparing commonly rated 
        elements of both users

        Returns
        -------
        float
            returns a float which measures the similarity of two user according 
            to their set of commonly given ratings, ranges from -1 to 1

        """
        sum_xy = 0
        sum_x = 0
        sum_y = 0
        sum_x2 = 0
        sum_y2 = 0
        n = 0
        for key in rating1:
            if key in rating2:
                n += 1
                x = rating1[key]
                y = rating2[key]
                sum_xy += x * y
                sum_x += x
                sum_y += y
                sum_x2 += pow(x, 2)
                sum_y2 += pow(y, 2)
        if n == 0:
            return 0
        # compute denominator
        denominator = (sqrt(sum_x2 - pow(sum_x, 2) / n)
                       * sqrt(sum_y2 - pow(sum_y, 2) / n))
        if denominator == 0:
            return 0
        else:
            return (sum_xy - (sum_x * sum_y) / n) / denominator

### Making sense of the similarities - KNN Approach
As previously mentioned, these similarity metrics are all used to measure to how similar users from given datasets are to one another. However, this is just one half of the collaborative filtering approach, as similarities by themselves are not sufficient in order to make the actual recommendation (and a possible prediction). In order to be able to recommend an item to a certain user, we must make use of one of the earliest machine learning algorithms - **K-nearest neighbors**. With KNN, we determine the $k$ nearest users to a given user with our chosen similarity metric, which means that ultimately we determine the k-most-similar users to an input user, analyze their ratings, calculate the respective contribution of each user to the final recommendation and predicted rating and return the recommendation with predicted rating as output. Naturally, the items that both users already rated fall out here, so we can only recommend items from the nearest neighbors that the user hasn't rated yet. Below is the implementation for both KNN and recommendation function:

In [ ]:
def nearest_neighbor(self, username):
        """
        Determines nearest neighbors, i.e. users for a user in a given dataset
        for collaborative filtering.
        Default metric is Pearson correlation coefficient, can be changed to other
        metrics when instatiating an object of Recommender class 

        Parameters
        ----------
        username : str
            username of a user in the set of our users
        
        since self.fn == 'pearson', the nearest neighbors will be calculated 
        using pearson coefficient, can be changed if desired

        Returns
        -------
        distances : list
            a list containing all the distances from the user to the other users
            in the total set of users
            Elements are tuples with users (str) as first element and the distances
            to them as floats as second element

        """
        distances = []
        for instance in self.data:
            # measure distance only if users are distinct 
            if instance != username:
                # Pearson metric is applied
                distance = self.fn(self.data[username], self.data[instance])
                distances.append((instance, distance))
        distances.sort(key=lambda artist_tuple: artist_tuple[1], reverse=True)
        return distances
    
    def recommend(self, user:str):
        """
        Recommends a 

        Parameters
        ----------
        user : str
            user in the set of users

        Returns
        -------
        type = list
            returns a list of top n recommendations for the user 

        """
        # create empty dictionary for recommendations
        recommendations = {}
        # compute the nearest neighbors of a specific user
        nearest = self.nearest_neighbor(user)
        # extract ratings for this specific user from self.data dict
        user_ratings = self.data[user]
        # initialize total distance counter
        total_distance = 0.0
        # iterate over k elements, here k=1
        for i in range(self.k):
            total_distance += nearest[i][1]
        # add up the total distance by summing over the ratings of k nearest neighbors
        # compute the contribution of each k neighbor
        for i in range(self.k):
            weight = nearest[i][1] / total_distance
            # extract name of each of the k neighbors
            name = nearest[i][0]
            # extract ratings of each person out of the k neighbors
            neighbor_ratings = self.data[name]
        # for every artist in neighbor ratings
        for artist in neighbor_ratings:
            # if the artist is not in the user_ratings
            if not artist in user_ratings:
                # and if the artist is not already in recommendations dict 
                # i.e. artists that the neigbor rated but the user didn't
                # for key artists, assign the value of its neighbor rating times the weight 
                # it contributes
                if artist not in recommendations:
                    # for key artists, assign the value of its neighbor rating times the weight 
                    # it contributes
                    recommendations[artist] = (neighbor_ratings[artist]*weight)
                else:
                    # if artist already existing, assign neighbor rating + recommendations rating
                    # and multiply it with weight
                    recommendations[artist] = (recommendations[artist]+neighbor_ratings[artist] * weight)
        # make a list out of the key, value pairs of artist and recommendations tuples
        recommendations = list(recommendations.items())
        # convert the product id to names via list comprehension
        recommendations = [(self.convertProductID2name(k), v) for (k, v) in recommendations]
        # sort the list by rating, reverse in order for top ratings to be at the beginning
        recommendations.sort(key=lambda artist_tuple: artist_tuple[1], reverse=True)
        # return n elements
        return recommendations[:self.n]

Collaborative Filtering thus provides a great way to create recommendations by following a very simple thought: An item that is liked by user will possibly be liked by similar users with similar tastes. As noted, it does have a few shortcomings such as absolute honesty from users when rating the items, or also the necessity of a considerably larger dataset of users (since we want to cover as similar users as possible), still, it appears to be very popular since it is also used by sites such as YouTube, where some video recommendations are marked with "users who have watched this also watched this video", and, as I will later show later on, it is also a popular base model for both *Neural Collaborative Filtering*. 

## Item-based filtering
Item-based filtering flipts the script and approaches the task of recommending from the perspective (and accesability) of the respective **items** that we rate and want to recommend. As Rob Zacharski suggests, the motivation behind item-based filtering is that users on certain platforms can often times be biased in their ratings and thus can fundamentally influence various different computations mentioned above for the worse. In order to overcome this phenomenon, data scientists have figured out that instead of performing various distance and similarity measures on users, it might be more suitable to perform these computations (with same databases as in collaborative filtering) on items instead. An especially brilliant method (which blows the proportions of this project, but is still worth mentioning) is **implicit ratings**, where users are not required to rate an item on any fixed scale; instead, other factors which don't require the user to perform any kind of direct rating such as *screen time for a certain item*, *searching frequency*, *number of times played/watched* etc. are taken into account. It is to be noted, however, that implicit ratings aren't free of flaws as well, since **outliers** in our patterns could skew the recommendations as well.<br>
In the class above, the task of item-based filtering is handled by *slope_one_recommender* function, which is based on the code from Rob Zacharski's *The Ancient Art of the Numerati*. I also added the *predict_rating* function which is based on adjusted cosine similarity and, as the name suggests, also returns a prediction for an item that is recommended to a user. In addition to that, I added the *nearest_items* function which works on the premise that items that are similar to the top rated items of a user might also be well received by the user, hence we recommend them to him/her. The logic behind this idea was that a user (depending on the platform) is likely to give the highest rating only to a few selected items. Since we already excluded the items that the user has already rated, finding out the most similar items to the highest rated items of a user gives us a good base of recommending items that a user might like, given the similarity to his favourite items. This can, however, be prone to outliers in the user's highest rated items, since an item of (for example) a completely different genre than the other items in his favourites could recommend items that are not necessarily the user's usual taste.

In [ ]:
 def compute_deviations(self):
        """
        Computes item deviations in a given dataset 
        
        args: self, no other argument necessary
        
        Functions computes deviations for each item according to the given 
        formula in Chapter 3 of "The ancient art of Numerati"
        
        After function is called, self.deviations will be filled with each item 
        as key, and its values are dictionaries in which keys represent other items 
        and values represent the deviation to that item in float type

        Returns
        -------
        None.

        """
        # for each person in the data :
        #get their ratings
        for ratings in self.data.values():
            #for each item and its rating in tthe ratings dict
            for (item, rating) in ratings.items():
                self.frequencies.setdefault(item, {})
                self.deviations.setdefault(item, {})
                # for each item2 & rating2 in that set of ratings: 
                for(item2, rating2) in ratings.items():
                    # add the difference between the ratings to our computation
                    if item != item2:
                        self.frequencies[item].setdefault(item2, 0)
                        self.deviations[item].setdefault(item2, 0.0)
                        self.frequencies[item][item2] += 1
                        self.deviations[item][item2] += rating - rating2
        for (item, ratings) in self.deviations.items():
            for item2 in ratings:
                ratings[item2] /= self.frequencies[item][item2]
    
        
    def slope_one_recommend(self, user):
        """
        Implementation of slope one algorithm for recommendation and 
        prediction of an item for a given user, for item-based recommendation

        Parameters
        ----------
        user : str
            user for which we will recommend items and predict his ratings
            self.compute_deviations function must be called before calling this 
            function
            

        Returns
        -------
        recommendations : list
            A list containing tuples as elements
            tuple[0] = item which user hasn't rated yet, type: str
            tuple[1] = predicted rating for that unrated item, type: float

        """
        self.compute_deviations()
        recommendations = {}
        frequencies = {}
        # for every item and rating in the user's recommendations
        for (user_item, user_rating) in self.data[user].items():
            # for every item in our dataset that the user didn't rate
            # example. dict_items([('Taylor Swift', 5), ('PSY', 2)])
            for (diff_item, diff_ratings) in self.deviations.items():
                if diff_item not in self.data[user] and \
                   user_item in self.deviations[diff_item]:
                # get the deviations, frequencies for the item that wasn't rated
                # but only if a value for deviations exists and diff_item is not 
                # in user_ratings
                    freq = self.frequencies[diff_item][user_item]
                    # get the frequency of diff_item and user_item, 
                    # key in frequencies is the different item, second dict indice 
                    # is the frequency with user_item
                    recommendations.setdefault(diff_item, 0.0)
                    frequencies.setdefault(diff_item, 0)
                    # add to the running sum representing the numerator
                    # of the formula 
                    # diff_item is key, result of formula is value
                    recommendations[diff_item] += (diff_ratings[user_item] +
                                                   user_rating)* freq
                    # keep a running sum of the frequency of diffitem
                    frequencies[diff_item] += freq
        recommendations = [(self.convertProductID2name(k), v / frequencies[k])
                           for (k, v) in recommendations.items()]
        recommendations.sort(key=lambda artist_tuple: artist_tuple[1], reverse = True)
        return recommendations
    
def item_similarity(self, artist1, artist2, user_ratings):
        """
        Computes the similarity between two items in a given dataset

        Parameters
        ----------
        artist1 : str
            artist1 for which we will compute the similarity with artist2, must be 
            in user_ratings dict
        artist2 : str
            artist1 for which we will compute the similarity with artist2, must be 
            in user_ratings dict    
        user_ratings : dict
            dictionary containing users as keys, their ratings for items 
            as internal dict as values

        Returns
        -------
        float
            similarity between two items

        """
        
        averages = {}
        for (key, ratings) in user_ratings.items():
            #take the values of the internal dict, sum over each value and divide by length of values
            averages[key] = (float(sum(ratings.values()))
                      / len(ratings.values()))
        numerator = 0
        denom1 = 0
        denom2 = 0
        for (user, ratings) in user_ratings.items():
            #if the users rated both artists
            if artist1 in ratings and artist2 in ratings:
                #compute the averages of each user
                avg = averages[user]
                # in ratings dict, artists are the keys and their respective values, 
                # which we access via indexing, are their ratings
                # those ratings for the artists are subtracted by the average of each user
                # and then multiplied
                numerator += (ratings[artist1] - avg) * (ratings[artist2] - avg)
                denom1 += (ratings[artist1]-avg)**2 
                denom2 += (ratings[artist2]-avg)**2
        return numerator / (sqrt(denom1) * sqrt(denom2))
    
def normalized_rating(self, user:str, item:str):
        """
        Normalizes the rating of an item for a user in a given dataset according 
        to minimum and maximum rating of that dataset

        Parameters
        ----------
        user : str
            a user who has rated the item function argument
        item : str
            item that a user has rated, is within the internal dictionary as key
        

        Returns
        -------
        float
            normalized rating for a user given said item

        """
        global min_rat
        global max_rat
        min_ratings = []
        max_ratings = []
        for (key, rating) in self.data.items():
            key_max = max(rating.keys(), key=(lambda k: rating[k]))
            key_min = min(rating.keys(), key=(lambda k: rating[k]))
            min_ratings.append(rating[key_min])
            max_ratings.append(rating[key_max])
        min_rat = min(min_ratings)
        max_rat = max(max_ratings)
        if item in self.data[user]:
            return (2*((self.data[user][item]-min_rat)) - (max_rat-min_rat)) / (max_rat-min_rat)
            
    
def denormalized_rating(self, user:str, item:str):
        """
        Returns a normalized rating back into a scaled rating, 
        given respective rating scale
        
        BEWARE!!! This function takes an user and an item and then gives the denormalized
        rating for it, the denormalizer only takes a float value

        Parameters
        ----------
        user : str
            DESCRIPTION.
        norm_rating : float
            DESCRIPTION.

        Returns
        -------
        None.

        """
        return (0.5*((self.normalized_rating(user, item)+1) * (max_rat-min_rat))) + min_rat
        
    
def denormalize(self, norm_rating:float):
        """
        Function for denormalizing a rating

        Parameters
        ----------
        norm_rating : float
            DESCRIPTION.

        Returns
        -------
        None.

        """
        return (0.5*((norm_rating+1) * (max_rat-min_rat))) + min_rat

        
def predict_rating(self, user:str, item:str):
        """
        Predicts the rating of an item from a user from a given dataset

        Parameters
        ----------
        user : str
            user in a given dataset
        item : str
            an item which the user HAS NOT rated yet

        Returns
        -------
        tuple
            tuple[0] = user 
            tuple[1] = item recommended for that user
            tuple[2] = predicted rating for that item

        """
        #initiate numerator and denumerator of formula
        denom = 0
        numerator = 0
        for rated_item in self.data[user]:
            if rated_item in self.data[user]:
                denom += (self.item_similarity(rated_item, item, self.data)
                          *self.normalized_rating(user, rated_item))
                numerator += abs(self.item_similarity(item, rated_item, self.data))
        return (user, item, self.denormalize(denom/numerator))

It is interesting to note that both Collaborative Filtering and Item-Based Recommending work mainly on the mathematical basis of **Similarity Measures** such as cosine distance, Pearson Correlation Coefficient etc., and are slighlty adapted to properly handle the task of recommending at hand. During my experiments with the manual datasets, I personally found Item-Based filtering more intriguing since it not only recommends items for a given user, but also predicts the rating (according to the parametrization one chooses) for these recommended items.<br>
Certain characteristics of Classical Recommenders include that the Data always has to be preprocessed in a way that it can be appropriately handled, i.e. choosing the right data format in associative arrays and filling them such that every rating can be associated to an item, with these arrays then again associated with a unique user. My experience also included an unsatsisfyingly long loading time for calculating the deviations, which is caused by the fact that the larger a dataset is the mor time all these mathematical calculations will take up time to calculate and return results (I waited for 30 Minutes until my MacBook crashed). It is also worth noting that classic recommender systems can only expand in their variety of similarity measures, since the data stays the same and cannot be e.g. embedded, which then again would enable other calculations to which I will come later in this notebook <br>
Because of this, I would personally deem classic recommender systems to be a good fit for smaller datasets or datasets which are can be easily split into smaller parts, at least if you don't have a computer strong enough to calculate necessary calculations in appropriate time.<br>
Item-based filtering attempts to circumnavigate the sheer unreliability of users, but since even item-based recommendations are somewhat linked to user-item interactions, they can't pass this barrier completely either.

## Comparing Neural Recommenders and Classic Recommenders
As can be seen here, the classic recommenders require more accurate detailed computations and a strong database in order to yield a feasible result, but if that's provided by implementation, than they are guaranteed to produce feasible results, except for the outliers where even they can't properly predict a good rating, if any rating at all. However, the versatility they offer via implementation of both collaborative filtering and item-based filtering is a great advantage if one were to desire to be able to recommend on the basis of both methods.<br>
Collaborative filtering with Neural Networks and Matrix Factorization, on the other hand, provides for an interesting method implementing and realizing the task of collaborative filtering as it primarily works with embeddings. However, my findings showed that Neural Collaborative Filtering depends on a strong database where as much of observations of each rating has to be given, since otherwise the predictions will be inaccurate to the model being biased towards ratings that is sees more often. Another problem is **Bias/Variance** since greatly overfitting the model will result in inaccurate predictions which thus negatively affects the final recommendations. The Neural Network model seemed to have the least problems with this, however, it also could have performed better, and given approprite conditions (e.g. strong CPU which can quickly calculate 1000 epochs), it could also provide better results. An advantage of Neural Models is that they are quite open to adjustment and extension; a model doesn't have to remain rigidly the same but one can experiment with many factors which could further improve the model's perfomance: hyperparameter, model architecture, embedding size etc. Every recommender has something in commmon, though: They each suffer from the tendency of Collaborative Filtering to recommend the most popular items and predict a high score them. Though this underlines the very notion of Collaborative Filtering actually, it is sometimes not quite what a powerful recommender should do, for there are users with peculiar tastes whose interests will not be satisfied by recommending them the most popular items. Still, for most usecases, (Neural) Collaborative Filtering should provide a good approach to the task of making effective recommendations for users of any given platform.